# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'QT A(SNBKERLGCEG DEAMSA LTANO)S/L',
 'calculationPrice': 'iexlasttrade',
 'open': None,
 'openTime': None,
 'openSource': 'icaioffl',
 'close': None,
 'closeTime': None,
 'closeSource': 'liafocfi',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 147.69,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'January 27, 2021',
 'latestUpdate': 1675758986085,
 'latestVolume': None,
 'iexRealtimePrice': 144.41,
 'iexRealtimeSize': 105,
 'iexLastUpdated': 1620362487558,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 147.54,
 'previousVolume': 100283721,
 'change': -0.69,
 'changePercent': -0.00484,
 'volume': None,
 'iexMarketPercent': 0.017115877531270512,
 'iexV

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

45.08

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,118.92,53.69,N/A
1,AAL,16.66,-1.22,N/A
2,AAP,154.86,22.42,N/A
3,AAPL,147.30,43.81,N/A
4,ABBV,105.08,22.93,N/A
...,...,...,...,...
500,YUM,104.32,30.64,N/A
501,ZBH,159.17,-212.22,N/A
502,ZBRA,408.96,45.7,N/A
503,ZION,45.17,19.25,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,39.830,2.5,N/A
1,NLOK,21.438,4.01,N/A
2,AIV,4.810,4.55,N/A
3,BIO,554.890,4.8,N/A
4,UNM,25.200,5.36,N/A
5,AFL,47.190,7.3,N/A
6,ALL,115.390,7.78,N/A
7,MET,48.594,8,N/A
8,EBAY,61.010,8.5,N/A
9,KIM,17.676,8.7,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,39.830,2.5,502
1,NLOK,21.438,4.01,932
2,AIV,4.810,4.55,4158
3,BIO,554.890,4.8,36
4,UNM,25.200,5.36,793
5,AFL,47.190,7.3,423
6,ALL,115.390,7.78,173
7,MET,48.594,8,411
8,EBAY,61.010,8.5,327
9,KIM,17.676,8.7,1131


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [12]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [14]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.150,N/A,53.77,N/A,8.1,N/A,7.32,N/A,34.238563,N/A,13.512909,N/A,N/A
1,AAL,17.188,N/A,-1.23,N/A,-1.8,N/A,0.3935,N/A,-6.168346,N/A,1.373907,N/A,N/A
2,AAP,149.720,N/A,22.05,N/A,2.97,N/A,1.11,N/A,10.733204,N/A,2.419472,N/A,N/A
3,AAPL,149.200,N/A,45.3,N/A,38.04,N/A,8.92,N/A,31.321411,N/A,22.635793,N/A,N/A
4,ABBV,102.880,N/A,22.92,N/A,12.89,N/A,4.9,N/A,15.791143,N/A,9.508382,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.390,N/A,30.93,N/A,-4.15,N/A,5.78,N/A,23.183185,N/A,9.872996,N/A,N/A
501,ZBH,153.810,N/A,-216.22,N/A,2.87,N/A,4.75,N/A,28.125245,N/A,8.204487,N/A,N/A
502,ZBRA,402.490,N/A,45.52,N/A,11.39,N/A,4.95,N/A,30.429271,N/A,11.603114,N/A,N/A
503,ZION,43.820,N/A,19.3,N/A,1.08,N/A,2.5,N/A,11.577369,N/A,2.999207,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,213.980,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,228.470,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.87,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,33.980,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.910,N/A,14.55,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,54.740,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,33.230,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,152.300,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1892.807,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.040,N/A,31.91,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [16]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [17]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [18]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0       0.867327
1       0.153465
2       0.453465
3       0.815842
4       0.471287
         ...    
500     0.655446
501    0.0257426
502     0.819802
503     0.405941
504     0.811881
Name: PE Percentile, Length: 505, dtype: object
0       0.746535
1      0.0475248
2        0.40297
3       0.966337
4       0.853465
         ...    
500    0.0435644
501     0.391089
502     0.833663
503     0.107921
504     0.926733
Name: PB Percentile, Length: 505, dtype: object
0        0.80396
1      0.0257426
2       0.148515
3       0.860396
4        0.69604
         ...    
500     0.742574
501     0.682178
502      0.69901
503     0.392079
504     0.934653
Name: PS Percentile, Length: 505, dtype: object
0      0.851485
1      0.029703
2      0.227723
3      0.835644
4      0.471287
         ...   
500     0.70297
501     0.79604
502    0.823762
503    0.279208
504    0.841584
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0        0.79802
1      0.0514851
2       0.112871
3       0.960

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.150,N/A,53.77,0.867327,8.10,0.746535,7.3200,0.80396,34.238563,0.851485,13.512909,0.79802,N/A
1,AAL,17.188,N/A,-1.23,0.153465,-1.80,0.0475248,0.3935,0.0257426,-6.168346,0.029703,1.373907,0.0514851,N/A
2,AAP,149.720,N/A,22.05,0.453465,2.97,0.40297,1.1100,0.148515,10.733204,0.227723,2.419472,0.112871,N/A
3,AAPL,149.200,N/A,45.30,0.815842,38.04,0.966337,8.9200,0.860396,31.321411,0.835644,22.635793,0.960396,N/A
4,ABBV,102.880,N/A,22.92,0.471287,12.89,0.853465,4.9000,0.69604,15.791143,0.471287,9.508382,0.645545,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.390,N/A,30.93,0.655446,-4.15,0.0435644,5.7800,0.742574,23.183185,0.70297,9.872996,0.659406,N/A
501,ZBH,153.810,N/A,-216.22,0.0257426,2.87,0.391089,4.7500,0.682178,28.125245,0.79604,8.204487,0.530693,N/A
502,ZBRA,402.490,N/A,45.52,0.819802,11.39,0.833663,4.9500,0.69901,30.429271,0.823762,11.603114,0.726733,N/A
503,ZION,43.820,N/A,19.30,0.405941,1.08,0.107921,2.5000,0.392079,11.577369,0.279208,2.999207,0.148515,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [19]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.150,N/A,53.77,0.867327,8.10,0.746535,7.3200,0.80396,34.238563,0.851485,13.512909,0.79802,0.813465
1,AAL,17.188,N/A,-1.23,0.153465,-1.80,0.0475248,0.3935,0.0257426,-6.168346,0.029703,1.373907,0.0514851,0.0615842
2,AAP,149.720,N/A,22.05,0.453465,2.97,0.40297,1.1100,0.148515,10.733204,0.227723,2.419472,0.112871,0.269109
3,AAPL,149.200,N/A,45.30,0.815842,38.04,0.966337,8.9200,0.860396,31.321411,0.835644,22.635793,0.960396,0.887723
4,ABBV,102.880,N/A,22.92,0.471287,12.89,0.853465,4.9000,0.69604,15.791143,0.471287,9.508382,0.645545,0.627525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.390,N/A,30.93,0.655446,-4.15,0.0435644,5.7800,0.742574,23.183185,0.70297,9.872996,0.659406,0.560792
501,ZBH,153.810,N/A,-216.22,0.0257426,2.87,0.391089,4.7500,0.682178,28.125245,0.79604,8.204487,0.530693,0.485149
502,ZBRA,402.490,N/A,45.52,0.819802,11.39,0.833663,4.9500,0.69901,30.429271,0.823762,11.603114,0.726733,0.780594
503,ZION,43.820,N/A,19.30,0.405941,1.08,0.107921,2.5000,0.392079,11.577369,0.279208,2.999207,0.148515,0.266733


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [20]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [21]:
portfolio_input()

Enter the value of your portfolio:1000000


In [22]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

c:\users\rkrit\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,38.580,518,-6.87,0.112871,0.5345,0.0574257,0.7715,0.0752475,4.396518,0.039604,0.733013,0.0138614,0.059802
1,AAL,17.188,1163,-1.23,0.153465,-1.8000,0.0475248,0.3935,0.0257426,-6.168346,0.029703,1.373907,0.0514851,0.0615842
2,UNM,24.700,809,5.40,0.19604,0.4711,0.0534653,0.4300,0.0316832,2.790969,0.0356436,0.429107,0.0039604,0.0641584
3,F,11.111,1800,-273.63,0.0237624,1.3800,0.175248,0.3390,0.0138614,4.589320,0.0435644,2.385328,0.110891,0.0734653
4,MET,49.875,401,8.24,0.20198,0.6176,0.0613861,0.7021,0.0613861,4.682084,0.0455446,0.695407,0.0118812,0.0764356
5,BA,197.294,101,-25.20,0.0752475,-9.6800,0.0336634,1.9100,0.311881,-28.415472,0.0178218,-196.171977,0.0019802,0.0881188
6,FTI,11.878,1683,-0.89,0.156436,1.1800,0.126733,0.3680,0.019802,3.308000,0.0376238,2.132901,0.10099,0.0883168
7,HIG,48.430,412,10.13,0.223762,1.0200,0.0970297,0.8780,0.0891089,4.473761,0.0415842,0.900590,0.0237624,0.0950495
8,ALL,114.300,174,7.91,0.2,1.3800,0.175248,0.7485,0.0693069,2.501829,0.0336634,0.739585,0.0158416,0.0988119
9,LNC,48.690,410,12.08,0.249505,0.4448,0.0514851,0.5544,0.0415842,9.183603,0.156436,0.543666,0.00990099,0.101782


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [23]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [26]:
writer.save()